In [4]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull
from sklearn.metrics import silhouette_score, adjusted_rand_score
from time import time


def load_and_display_obj(file_path):
    # Read and visualize the 3D object
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    # Reduce dimensions of the points in the mesh using PCA
    points = mesh.points
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)

    # Visualize the reduced points
    reduced_cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(reduced_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return reduced_points, pca

def filter_convex_hull(points):
    # Compute and filter points on the convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Visualize convex hull points
    hull_cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(hull_cloud, color='blue', render_points_as_spheres=True, point_size=5)
    plotter.show()

    return hull_points

def fit_mixture_of_gaussians(points, n_components=4):
    # Apply Gaussian Mixture Model to fit and cluster points
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Visualize GMM clustering results
    clustered_cloud = pv.PolyData(points)
    clustered_cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(clustered_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return labels, gmm

def detect_and_print_primitives(points, labels):
    """
    Detect geometric primitives from clusters and print their shape.
    """
    unique_labels = np.unique(labels)
    detected_shapes = []

    for label in unique_labels:
        cluster_points = points[labels == label]

        # Analyze cluster properties (example: bounding box dimensions)
        bounds = np.ptp(cluster_points, axis=0)  # Peak-to-peak (range) along each axis
        dimensions = bounds / np.max(bounds)  # Normalize dimensions to compare ratios

        if np.allclose(dimensions, [1, 1, 1], atol=0.2):  # Close to cubic dimensions
            shape = "Sphere or Cube"
        elif dimensions[0] > dimensions[1] and dimensions[0] > dimensions[2]:
            shape = "Cylinder-like (elongated along X)"
        elif dimensions[1] > dimensions[0] and dimensions[1] > dimensions[2]:
            shape = "Cylinder-like (elongated along Y)"
        elif dimensions[2] > dimensions[0] and dimensions[2] > dimensions[1]:
            shape = "Cylinder-like (elongated along Z)"
        else:
            shape = "Irregular/Unknown"

        detected_shapes.append(shape)

        print(f"Cluster {label}: Detected Shape - {shape}")

    return detected_shapes

def generate_grasp_plan(points, labels):
    # Visualize grasp plans for detected primitives
    cluster_centroids = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        cluster_points = points[labels == label]
        centroid = np.mean(cluster_points, axis=0)

        # Ensure centroids are of float type for further processing
        centroid = np.array(centroid, dtype=np.float64)  # Convert to float64 explicitly
        cluster_centroids.append(centroid)

        # Visualize each centroid
        # sphere = pv.Sphere(center=centroid, radius=0.02)
        # plotter = pv.Plotter()
        # plotter.add_mesh(sphere, color="red")
        # plotter.show()

    return cluster_centroids

def evaluate_clustering(points, true_labels=None):
    # Fit GMM
    gmm = GaussianMixture(n_components=4, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Silhouette Score
    silhouette = silhouette_score(points, labels)
    print(f"Silhouette Score: {silhouette}")

    def measure_time(func, *args):
        start_time = time()
        result = func(*args)
        end_time = time()
        print(f"Execution Time: {end_time - start_time} seconds")
        return result

def main():
    obj_file_path = "../obj/bottle.obj"

    # Step 1: Load and visualize the 3D object
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions
    reduced_points, _ = reduce_dimensions(mesh)

    # Step 3: Filter convex hull points
    hull_points = filter_convex_hull(reduced_points)

    # Step 4: Fit Gaussian Mixture Model and visualize clusters
    labels, gmm = fit_mixture_of_gaussians(hull_points, n_components=4)

    # Step 5: Detect and print primitive shapes
    detected_shapes = detect_and_print_primitives(hull_points, labels)

    # Step 6: Generate and visualize grasp plans
    centroids = generate_grasp_plan(hull_points, labels)
    for idx, centroid in enumerate(centroids):
        print(f"Grasp Plan {idx + 1}: Centroid at {centroid}")
    
    evaluate_clustering(hull_points)
    
    

if __name__ == "__main__":
    main()

Widget(value='<iframe src="http://localhost:49821/index.html?ui=P_0x16566c920_8&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:49821/index.html?ui=P_0x166a1b590_9&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:49821/index.html?ui=P_0x166a04a40_10&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:49821/index.html?ui=P_0x166a19220_11&reconnect=auto" class="pyvist…

Cluster 0: Detected Shape - Cylinder-like (elongated along Z)
Cluster 1: Detected Shape - Cylinder-like (elongated along Z)
Cluster 2: Detected Shape - Cylinder-like (elongated along Z)
Cluster 3: Detected Shape - Cylinder-like (elongated along Z)
Grasp Plan 1: Centroid at [ 1.08875677e-02 -1.40581724e-05  6.96515766e-06]
Grasp Plan 2: Centroid at [-0.13310048 -0.01764257  0.01608137]
Grasp Plan 3: Centroid at [-0.13549603  0.01225103 -0.01137257]
Grasp Plan 4: Centroid at [ 5.91160672e-02 -2.03819637e-05  9.17874230e-06]
Silhouette Score: 0.52333402324355


In [24]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull

def load_and_display_obj(file_path):
    # Read and visualize the 3D object
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    # Reduce dimensions of the points in the mesh using PCA
    points = mesh.points
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)

    # Visualize the reduced points
    reduced_cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(reduced_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return reduced_points, pca

def filter_convex_hull(points):
    # Compute and filter points on the convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Visualize convex hull points
    hull_cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(hull_cloud, color='blue', render_points_as_spheres=True, point_size=5)
    plotter.show()

    return hull_points

def fit_mixture_of_gaussians(points, n_components=4):
    # Apply Gaussian Mixture Model to fit and cluster points
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Visualize GMM clustering results
    clustered_cloud = pv.PolyData(points)
    clustered_cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(clustered_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return labels, gmm

def detect_and_print_primitives(points, labels):
    """
    Detect geometric primitives from clusters and print their shape.
    """
    unique_labels = np.unique(labels)
    detected_shapes = []

    for label in unique_labels:
        cluster_points = points[labels == label]

        # Analyze cluster properties (example: bounding box dimensions)
        bounds = np.ptp(cluster_points, axis=0)  # Peak-to-peak (range) along each axis
        dimensions = bounds / np.max(bounds)  # Normalize dimensions to compare ratios

        if np.allclose(dimensions, [1, 1, 1], atol=0.2):  # Close to cubic dimensions
            shape = "Sphere or Cube"
        elif dimensions[0] > dimensions[1] and dimensions[0] > dimensions[2]:
            shape = "Cylinder-like (elongated along X)"
        elif dimensions[1] > dimensions[0] and dimensions[1] > dimensions[2]:
            shape = "Cylinder-like (elongated along Y)"
        elif dimensions[2] > dimensions[0] and dimensions[2] > dimensions[1]:
            shape = "Cylinder-like (elongated along Z)"
        else:
            shape = "Irregular/Unknown"

        detected_shapes.append(shape)

        print(f"Cluster {label}: Detected Shape - {shape}")

    return detected_shapes

def generate_grasp_plan(points, labels):
    # Visualize grasp plans for detected primitives
    cluster_centroids = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        cluster_points = points[labels == label]
        centroid = np.mean(cluster_points, axis=0)

        # Ensure centroids are of float type for further processing
        centroid = np.array(centroid, dtype=np.float64)  # Convert to float64 explicitly
        cluster_centroids.append(centroid)

        # Visualize each centroid
        # sphere = pv.Sphere(center=centroid, radius=0.02)
        # plotter = pv.Plotter()
        # plotter.add_mesh(sphere, color="red")
        # plotter.show()

    return cluster_centroids

def main():
    obj_file_path = "../obj/cup.obj"

    # Step 1: Load and visualize the 3D object
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions
    reduced_points, _ = reduce_dimensions(mesh)

    # Step 3: Filter convex hull points
    hull_points = filter_convex_hull(reduced_points)

    # Step 4: Fit Gaussian Mixture Model and visualize clusters
    labels, gmm = fit_mixture_of_gaussians(hull_points, n_components=4)

    # Step 5: Detect and print primitive shapes
    detected_shapes = detect_and_print_primitives(hull_points, labels)

    # Step 6: Generate and visualize grasp plans
    centroids = generate_grasp_plan(hull_points, labels)
    for idx, centroid in enumerate(centroids):
        print(f"Grasp Plan {idx + 1}: Centroid at {centroid}")

if __name__ == "__main__":
    main()

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x48d5bc830_93&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x4945584d0_94&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x3d1c03da0_95&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x44224b170_96&reconnect=auto" class="pyvist…

Cluster 0: Detected Shape - Cylinder-like (elongated along Z)
Cluster 1: Detected Shape - Cylinder-like (elongated along Y)
Cluster 2: Detected Shape - Cylinder-like (elongated along Z)
Cluster 3: Detected Shape - Cylinder-like (elongated along Y)
Cluster 4: Detected Shape - Cylinder-like (elongated along Z)
Cluster 5: Detected Shape - Cylinder-like (elongated along X)
Cluster 6: Detected Shape - Cylinder-like (elongated along Z)
Cluster 7: Detected Shape - Cylinder-like (elongated along Y)
Cluster 8: Detected Shape - Cylinder-like (elongated along Y)
Cluster 9: Detected Shape - Cylinder-like (elongated along X)
Grasp Plan 1: Centroid at [-0.21851411 -0.17278626  0.02316174]
Grasp Plan 2: Centroid at [0.26082895 0.06617371 0.19170542]
Grasp Plan 3: Centroid at [-0.30350109  0.30433654  0.0778679 ]
Grasp Plan 4: Centroid at [ 0.24952164  0.15777308 -0.2136264 ]
Grasp Plan 5: Centroid at [ 0.26446701 -0.09749895 -0.03322297]
Grasp Plan 6: Centroid at [ 0.15989639 -0.27784344 -0.00222758]

In [ ]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull

def load_and_display_obj(file_path):
    # Read and visualize the 3D object
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    # Reduce dimensions of the points in the mesh using PCA
    points = mesh.points
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)

    # Visualize the reduced points
    reduced_cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(reduced_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return reduced_points, pca

def filter_convex_hull(points):
    # Compute and filter points on the convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Visualize convex hull points
    hull_cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(hull_cloud, color='blue', render_points_as_spheres=True, point_size=5)
    plotter.show()

    return hull_points

def fit_mixture_of_gaussians(points, n_components=4):
    # Apply Gaussian Mixture Model to fit and cluster points
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Visualize GMM clustering results
    clustered_cloud = pv.PolyData(points)
    clustered_cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(clustered_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return labels, gmm

def detect_and_print_primitives(points, labels):
    """
    Detect geometric primitives from clusters and print their shape.
    """
    unique_labels = np.unique(labels)
    detected_shapes = []

    for label in unique_labels:
        cluster_points = points[labels == label]

        # Analyze cluster properties (example: bounding box dimensions)
        bounds = np.ptp(cluster_points, axis=0)  # Peak-to-peak (range) along each axis
        dimensions = bounds / np.max(bounds)  # Normalize dimensions to compare ratios

        if np.allclose(dimensions, [1, 1, 1], atol=0.2):  # Close to cubic dimensions
            shape = "Sphere or Cube"
        elif dimensions[0] > dimensions[1] and dimensions[0] > dimensions[2]:
            shape = "Cylinder-like (elongated along X)"
        elif dimensions[1] > dimensions[0] and dimensions[1] > dimensions[2]:
            shape = "Cylinder-like (elongated along Y)"
        elif dimensions[2] > dimensions[0] and dimensions[2] > dimensions[1]:
            shape = "Cylinder-like (elongated along Z)"
        else:
            shape = "Irregular/Unknown"

        detected_shapes.append(shape)

        print(f"Cluster {label}: Detected Shape - {shape}")

    return detected_shapes

def generate_grasp_plan(points, labels):
    # Visualize grasp plans for detected primitives
    cluster_centroids = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        cluster_points = points[labels == label]
        centroid = np.mean(cluster_points, axis=0)

        # Ensure centroids are of float type for further processing
        centroid = np.array(centroid, dtype=np.float64)  # Convert to float64 explicitly
        cluster_centroids.append(centroid)

        # Visualize each centroid
        # sphere = pv.Sphere(center=centroid, radius=0.02)
        # plotter = pv.Plotter()
        # plotter.add_mesh(sphere, color="red")
        # plotter.show()

    return cluster_centroids

def main():
    obj_file_path = "../obj/lamp.obj"
    
    # Step 1: Load and visualize the 3D object
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions
    reduced_points, _ = reduce_dimensions(mesh)

    # Step 3: Filter convex hull points
    hull_points = filter_convex_hull(reduced_points)

    # Step 4: Fit Gaussian Mixture Model and visualize clusters
    labels, gmm = fit_mixture_of_gaussians(hull_points, n_components=4)

    # Step 5: Detect and print primitive shapes
    detected_shapes = detect_and_print_primitives(hull_points, labels)

    # Step 6: Generate and visualize grasp plans
    centroids = generate_grasp_plan(hull_points, labels)
    for idx, centroid in enumerate(centroids):
        print(f"Grasp Plan {idx + 1}: Centroid at {centroid}")

if __name__ == "__main__":
    main()

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x36593c860_36&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x36947eea0_37&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x36a043620_38&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x374c35fa0_39&reconnect=auto" class="pyvist…

Cluster 0: Detected Shape - Cylinder-like (elongated along Z)
Cluster 1: Detected Shape - Cylinder-like (elongated along Z)
Cluster 2: Detected Shape - Cylinder-like (elongated along Z)
Cluster 3: Detected Shape - Cylinder-like (elongated along Z)
Grasp Plan 1: Centroid at [0.25385236 0.12572917 0.04761816]
Grasp Plan 2: Centroid at [-0.42673077  0.14696336  0.01328166]
Grasp Plan 3: Centroid at [ 0.12131841 -0.04201496 -0.04200226]
Grasp Plan 4: Centroid at [-0.08094122 -0.23375066  0.03615692]


In [ ]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull

def load_and_display_obj(file_path):
    # Read and visualize the 3D object
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    # Reduce dimensions of the points in the mesh using PCA
    points = mesh.points
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)

    # Visualize the reduced points
    reduced_cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(reduced_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return reduced_points, pca

def filter_convex_hull(points):
    # Compute and filter points on the convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Visualize convex hull points
    hull_cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(hull_cloud, color='blue', render_points_as_spheres=True, point_size=5)
    plotter.show()

    return hull_points

def fit_mixture_of_gaussians(points, n_components=4):
    # Apply Gaussian Mixture Model to fit and cluster points
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Visualize GMM clustering results
    clustered_cloud = pv.PolyData(points)
    clustered_cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(clustered_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return labels, gmm

def detect_and_print_primitives(points, labels):
    """
    Detect geometric primitives from clusters and print their shape.
    """
    unique_labels = np.unique(labels)
    detected_shapes = []

    for label in unique_labels:
        cluster_points = points[labels == label]

        # Analyze cluster properties (example: bounding box dimensions)
        bounds = np.ptp(cluster_points, axis=0)  # Peak-to-peak (range) along each axis
        dimensions = bounds / np.max(bounds)  # Normalize dimensions to compare ratios

        if np.allclose(dimensions, [19, 19, 19], atol=0.02):  # Close to cubic dimensions
            shape = "Sphere or Cube"
        elif dimensions[0] > dimensions[1] and dimensions[0] > dimensions[2]:
            shape = "Cylinder-like (elongated along X)"
        elif dimensions[1] > dimensions[0] and dimensions[1] > dimensions[2]:
            shape = "Cylinder-like (elongated along Y)"
        elif dimensions[2] > dimensions[0] and dimensions[2] > dimensions[1]:
            shape = "Cylinder-like (elongated along Z)"
        else:
            shape = "Irregular/Unknown"

        detected_shapes.append(shape)

        print(f"Cluster {label}: Detected Shape - {shape}")

    return detected_shapes

def generate_grasp_plan(points, labels):
    # Visualize grasp plans for detected primitives
    cluster_centroids = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        cluster_points = points[labels == label]
        centroid = np.mean(cluster_points, axis=0)

        # Ensure centroids are of float type for further processing
        centroid = np.array(centroid, dtype=np.float64)  # Convert to float64 explicitly
        cluster_centroids.append(centroid)

        # Visualize each centroid
        # sphere = pv.Sphere(center=centroid, radius=0.02)
        # plotter = pv.Plotter()
        # plotter.add_mesh(sphere, color="red")
        # plotter.show()

    return cluster_centroids

def main():
    obj_file_path = "../obj/cube.obj"
    
    # Step 1: Load and visualize the 3D object
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions
    reduced_points, _ = reduce_dimensions(mesh)

    # Step 3: Filter convex hull points
    hull_points = filter_convex_hull(reduced_points)

    # Step 4: Fit Gaussian Mixture Model and visualize clusters
    labels, gmm = fit_mixture_of_gaussians(hull_points, n_components=4)

    # Step 5: Detect and print primitive shapes
    detected_shapes = detect_and_print_primitives(hull_points, labels)

    # Step 6: Generate and visualize grasp plans
    centroids = generate_grasp_plan(hull_points, labels)
    for idx, centroid in enumerate(centroids):
        print(f"Grasp Plan {idx + 1}: Centroid at {centroid}")

if __name__ == "__main__":
    main()

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x3b367b260_72&reconnect=auto" class="pyvist…

Context leak detected, msgtracer returned -1


Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x442b29580_73&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x47b46b2c0_74&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x47b46b1d0_75&reconnect=auto" class="pyvist…

Cluster 0: Detected Shape - Cylinder-like (elongated along Y)
Cluster 1: Detected Shape - Cylinder-like (elongated along Y)
Cluster 2: Detected Shape - Cylinder-like (elongated along Z)
Cluster 3: Detected Shape - Irregular/Unknown
Grasp Plan 1: Centroid at [-1.20710728 -0.70710642 -0.20710567]
Grasp Plan 2: Centroid at [0.20710725 0.70710856 1.20710587]
Grasp Plan 3: Centroid at [ 0.90236853 -0.47140566 -0.4309639 ]
Grasp Plan 4: Centroid at [-0.7071055   1.41421278 -0.70710877]


/var/folders/pm/_y3ccgc55js8fzl2fwr1p2nr0000gn/T/ipykernel_56318/57105569.py:69: RuntimeWarning: invalid value encountered in divide
  dimensions = bounds / np.max(bounds)  # Normalize dimensions to compare ratios


In [ ]:
import numpy as np
import pyvista as pv
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from scipy.spatial import ConvexHull

def load_and_display_obj(file_path):
    # Read and visualize the 3D object
    mesh = pv.read(file_path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='white', show_edges=True)
    plotter.show()
    return mesh

def reduce_dimensions(mesh, n_components=3):
    # Reduce dimensions of the points in the mesh using PCA
    points = mesh.points
    pca = PCA(n_components=n_components)
    reduced_points = pca.fit_transform(points)

    # Visualize the reduced points
    reduced_cloud = pv.PolyData(reduced_points)
    plotter = pv.Plotter()
    plotter.add_mesh(reduced_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return reduced_points, pca

def filter_convex_hull(points):
    # Compute and filter points on the convex hull
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]

    # Visualize convex hull points
    hull_cloud = pv.PolyData(hull_points)
    plotter = pv.Plotter()
    plotter.add_mesh(hull_cloud, color='blue', render_points_as_spheres=True, point_size=5)
    plotter.show()

    return hull_points

def fit_mixture_of_gaussians(points, n_components=2):
    # Apply Gaussian Mixture Model to fit and cluster points
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(points)
    labels = gmm.predict(points)

    # Visualize GMM clustering results
    clustered_cloud = pv.PolyData(points)
    clustered_cloud["Cluster"] = labels
    plotter = pv.Plotter()
    plotter.add_mesh(clustered_cloud, render_points_as_spheres=True, point_size=5)
    plotter.show()

    return labels, gmm

def detect_and_print_primitives(points, labels):
    """
    Detect geometric primitives from clusters and print their shape.
    """
    unique_labels = np.unique(labels)
    detected_shapes = []

    for label in unique_labels:
        cluster_points = points[labels == label]

        # Analyze cluster properties (example: bounding box dimensions)
        bounds = np.ptp(cluster_points, axis=0)  # Peak-to-peak (range) along each axis
        dimensions = bounds / np.max(bounds)  # Normalize dimensions to compare ratios

        if np.allclose(dimensions, [1, 1, 1], atol=0.2):  # Close to cubic dimensions
            shape = "Sphere or Cube"
        elif dimensions[0] > dimensions[1] and dimensions[0] > dimensions[2]:
            shape = "Cylinder-like (elongated along X)"
        elif dimensions[1] > dimensions[0] and dimensions[1] > dimensions[2]:
            shape = "Cylinder-like (elongated along Y)"
        elif dimensions[2] > dimensions[0] and dimensions[2] > dimensions[1]:
            shape = "Cylinder-like (elongated along Z)"
        else:
            shape = "Irregular/Unknown"

        detected_shapes.append(shape)

        print(f"Cluster {label}: Detected Shape - {shape}")

    return detected_shapes

def generate_grasp_plan(points, labels):
    # Visualize grasp plans for detected primitives
    cluster_centroids = []
    unique_labels = np.unique(labels)

    for label in unique_labels:
        cluster_points = points[labels == label]
        centroid = np.mean(cluster_points, axis=0)

        # Ensure centroids are of float type for further processing
        centroid = np.array(centroid, dtype=np.float64)  # Convert to float64 explicitly
        cluster_centroids.append(centroid)

        # Visualize each centroid
        # sphere = pv.Sphere(center=centroid, radius=0.02)
        # plotter = pv.Plotter()
        # plotter.add_mesh(sphere, color="red")
        # plotter.show()

    return cluster_centroids

def main():
    obj_file_path = "../obj/ball/Ball.obj"

    # Step 1: Load and visualize the 3D object
    mesh = load_and_display_obj(obj_file_path)

    # Step 2: Reduce dimensions
    reduced_points, _ = reduce_dimensions(mesh)

    # Step 3: Filter convex hull points
    hull_points = filter_convex_hull(reduced_points)

    # Step 4: Fit Gaussian Mixture Model and visualize clusters
    labels, gmm = fit_mixture_of_gaussians(hull_points, n_components=4)

    # Step 5: Detect and print primitive shapes
    detected_shapes = detect_and_print_primitives(hull_points, labels)

    # Step 6: Generate and visualize grasp plans
    centroids = generate_grasp_plan(hull_points, labels)
    for idx, centroid in enumerate(centroids):
        print(f"Grasp Plan {idx + 1}: Centroid at {centroid}")

if __name__ == "__main__":
    main()

2025-01-20 23:17:47.520 (2251.717s) [          176488]       vtkOBJReader.cxx:215   WARN| vtkOBJReader (0x3dc9f2ff0): unexpected data at end of line in OBJ file L.3


Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x442b298e0_85&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x442216090_86&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x44222a300_87&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:60531/index.html?ui=P_0x4422484d0_88&reconnect=auto" class="pyvist…

Cluster 0: Detected Shape - Cylinder-like (elongated along X)
Cluster 1: Detected Shape - Cylinder-like (elongated along Z)
Cluster 2: Detected Shape - Cylinder-like (elongated along Y)
Cluster 3: Detected Shape - Sphere or Cube
Grasp Plan 1: Centroid at [ 0.08519437 -0.6971799  -0.16843825]
Grasp Plan 2: Centroid at [-0.70062766  0.17371759 -0.04030424]
Grasp Plan 3: Centroid at [0.2378867  0.1164757  0.67224966]
Grasp Plan 4: Centroid at [ 0.37602128  0.40705684 -0.46329642]
